# Modelo Recomendador

In [ ]:
# Importamos todas las librerías necesarias
import pandas as pd
import numpy as np
from math import radians, cos, sin, asin, sqrt
!pip install thefuzz[speedup]
from thefuzz import process
import gradio as gr
import folium
from folium.plugins import MarkerCluster
import requests
import base64

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 34.8 MB/s eta 0:00:00


In [ ]:
# Cargamos el datset
df = pd.read_excel('general_7.xlsx')

In [ ]:
# Mostramos las primeras filas del DataFrame
df.head()

,Nombre municipio,Codigo,COD_INE,Nombre,Provincia,LONGITUD_ETRS89,LATITUD_ETRS89,Habitantes,Num_Hospitales,Num_Centros_AP,Num_Infantil,Num_Primaria,Num_Secundaria,Precio_Alquiler,Precio_Venta
0,44001 Ababuj,44001,44001000000,Ababuj,Teruel,-0.807801,40.548459,74,0,1,0,0,0,8.100000,1205.000000
1,40001 Abades,40001,40001000000,Abades,Segovia,-4.267874,40.916340,859,0,1,1,0,0,NaN,1493.666667
2,10001 Abadía,10001,10001000000,Abadía,Cáceres,-5.977858,40.259532,331,0,1,0,0,0,7.700000,916.533333
3,27001 Abadín,27001,27001000000,Abadín,Lugo,-7.474996,43.363587,2217,0,1,1,1,0,7.900000,1141.111111
4,48001 Abadiño,48001,48001000000,Abadiño,Bizkaia,-2.606873,43.147646,7742,0,2,0,0,0,14.433333,2705.052632


In [ ]:
# Mostramos información general del DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8138 entries, 0 to 8137
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Nombre municipio  8138 non-null   object 
 1   Codigo            8138 non-null   int64  
 2   COD_INE           8138 non-null   int64  
 3   Nombre            8138 non-null   object 
 4   Provincia         8132 non-null   object 
 5   LONGITUD_ETRS89   8132 non-null   float64
 6   LATITUD_ETRS89    8132 non-null   float64
 7   Habitantes        8138 non-null   object 
 8   Num_Hospitales    8138 non-null   int64  
 9   Num_Centros_AP    8138 non-null   int64  
 10  Num_Infantil      8138 non-null   int64  
 11  Num_Primaria      8138 non-null   int64  
 12  Num_Secundaria    8138 non-null   int64  
 13  Precio_Alquiler   7700 non-null   float64
 14  Precio_Venta      7746 non-null   float64
dtypes: float64(4), int64(7), object(4)
memory usage: 953.8+ KB


In [ ]:
#Convertimos a número entero la variable Habitantes
df['Habitantes'] = pd.to_numeric(df['Habitantes'], errors='coerce')

In [ ]:
# Función para calcular distancia Haversine en km entre dos puntos lat/lon
def haversine(lon1, lat1, lon2, lat2):
    # Convertimos coordenadas a radianes
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # Diferencias de coordenadas
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    # Fórmula Haversine
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371  # Radio de la Tierra en km
    return c * r  # Distancia en km

# Parámetros de entrada del usuario
localidad_input = 'Barcelona'  # Municipio origen
provincia_input = 'Barcelona'  # Provincia origen
modalidad_trabajo = 'híbrido'  # Modalidad laboral

# Filtros establecidos por el usuario
distancia_max_km = 200  # Distancia máxima al origen
rango_precio_alquiler = (0, 30)  # €/m² de alquiler
rango_precio_venta = (0, 5000)   # €/m² de venta

# Requisitos del usuario sobre servicios y entorno
requisitos = {
    'cap': True,             # Centro de atención primaria
    'hospital': False,       # Hospital no requerido
    'infantil': False,       # Escuela infantil no requerida
    'primaria': True,        # Escuela primaria requerida
    'secundaria': False,     # Secundaria no requerida
    'entorno_rural': False,  # No quiere entorno rural
}

# Pesos asignados por el usuario a cada criterio
# Valor 0: No me importa
# Valor 1: Poco importante
# Valor 3: Importante
# Valor 5: Muy importante / clave
pesos_usuario = {
    'distancia': 4,
    'precio_alquier': 5,
    'precio_venta': 4,
    'cap': 3,
    'hospital': 4,
    'infantil': 2,
    'primaria': 3,
    'secundaria': 2,
    'entorno_rural': 1,
}


In [ ]:
#Ejemplo ilustrativo para mostrar cómo funciona thefuzz
#Texto con posible error ortográfico
localidad_input_prueba = "Madríd"

# Lista de nombres válidos
lista_nombres = ["Madrid", "Málaga", "Murcia", "Mallorca"]

# Obtenemos la mejor coincidencia y su puntuación
mejor_coincidencia, score = process.extractOne(localidad_input_prueba, lista_nombres)

# Mostramos la mejor coincidencia
print(mejor_coincidencia)

# Mostramos la puntuación de similitud
print(score)


Madrid
91


In [ ]:
#Buscamos la localidad de la entrada del usuario en nuestro datset para encontrar las coordenadas

# Filtramos por provincia (ignora mayúsculas/minúsculas)
df_provincia = df[df['Provincia'].str.lower() == provincia_input.lower()]

# Extraemos lista de localidades de esa provincia
lista_municipios = df_provincia['Nombre'].tolist()

# Buscamos la mejor coincidencia con fuzzy matching
localidad_match, score = process.extractOne(localidad_input, lista_municipios)

# Mostramos localidad original
print(f"Localidad ingresada: {localidad_input}")

# Mostramos localidad encontrada y nivel de coincidencia
print(f"Localidad econtrada: {localidad_match} (confianza {score}%)")


Localidad ingresada: Barcelona
Localidad econtrada: Barcelona (confianza 100%)


In [ ]:
# Obtenemos la fila de la localidad encontrada
origen = df_provincia[df_provincia['Nombre'] == localidad_match].iloc[0]

# Extraemos  longitud y latitud
lon_origen = origen['LONGITUD_ETRS89']
lat_origen = origen['LATITUD_ETRS89']

# Mostramos las coordenadas
print(lon_origen,lat_origen)

2.17634927 41.38424664


In [ ]:
# Añadimos columna con distancia en km desde el origen
df['Distancia_km'] = df.apply(lambda row: haversine(lon_origen, lat_origen, row['LONGITUD_ETRS89'], row['LATITUD_ETRS89']), axis=1)

# Calculamos tiempo en coche estimado (asumiendo 100 km/h)
df['Tiempo_coche_min'] = df['Distancia_km'] / 100 * 60


In [ ]:
# Clasificamos si es rural (menos de 10,000 habitantes)
df['Es_rural'] = df['Habitantes'] < 10000


In [ ]:
#Comprovamos que hemos añadido las nuevas columnas en nuestro dataframe
df.head()

,Nombre municipio,Codigo,COD_INE,Nombre,Provincia,LONGITUD_ETRS89,LATITUD_ETRS89,Habitantes,Num_Hospitales,Num_Centros_AP,Num_Infantil,Num_Primaria,Num_Secundaria,Precio_Alquiler,Precio_Venta,Distancia_km,Tiempo_coche_min,Es_rural
0,44001 Ababuj,44001,44001000000,Ababuj,Teruel,-0.807801,40.548459,74.0,0,1,0,0,0,8.100000,1205.000000,267.219451,160.331671,True
1,40001 Abades,40001,40001000000,Abades,Segovia,-4.267874,40.916340,859.0,0,1,1,0,0,NaN,1493.666667,541.938535,325.163121,True
2,10001 Abadía,10001,10001000000,Abadía,Cáceres,-5.977858,40.259532,331.0,0,1,0,0,0,7.700000,916.533333,697.171462,418.302877,True
3,27001 Abadín,27001,27001000000,Abadín,Lugo,-7.474996,43.363587,2217.0,0,1,1,1,0,7.900000,1141.111111,822.264097,493.358458,True
4,48001 Abadiño,48001,48001000000,Abadiño,Bizkaia,-2.606873,43.147646,7742.0,0,2,0,0,0,14.433333,2705.052632,439.642754,263.785653,True


In [ ]:
# Filtramos localidades dentro de la distancia máxima
df_filtrado = df[df['Distancia_km'] <= distancia_max_km]

# Excluimos la localidad de origen del filtro
df_filtrado = df_filtrado[df_filtrado['Nombre'].str.lower() != localidad_match.lower()]

# Filtramos por rango de precio de alquiler
df_filtrado = df_filtrado[
    (df_filtrado['Precio_Alquiler'] >= rango_precio_alquiler[0]) &
    (df_filtrado['Precio_Alquiler'] <= rango_precio_alquiler[1])
]

# Filtramos por rango de precio de venta
df_filtrado = df_filtrado[
    (df_filtrado['Precio_Venta'] >= rango_precio_venta[0]) &
    (df_filtrado['Precio_Venta'] <= rango_precio_venta[1])
]


In [ ]:
# Filtrar según requisitos de centros y entorno

if requisitos['cap']:
    # Filtramos localidades con al menos un centro de atención primaria
    df_filtrado = df_filtrado[df_filtrado['Num_Centros_AP'] > 0]

if requisitos['hospital']:
    # Filtramos localidades  con al menos un hospital
    df_filtrado = df_filtrado[df_filtrado['Num_Hospitales'] > 0]

if requisitos['infantil']:
    # Filtramos localidades con al menos una escuela infantil
    df_filtrado = df_filtrado[df_filtrado['Num_Infantil'] > 0]

if requisitos['primaria']:
    # Filtramos localidades  con al menos una escuela primaria
    df_filtrado = df_filtrado[df_filtrado['Num_Primaria'] > 0]

if requisitos['secundaria']:
    # Filtramos localidades  con al menos una escuela secundaria
    df_filtrado = df_filtrado[df_filtrado['Num_Secundaria'] > 0]

if requisitos['entorno_rural']:
    # Filtramos localidades  que son rurales
    df_filtrado = df_filtrado[df_filtrado['Es_rural'] == True]


In [ ]:
# Definimos criterios base y filtrar pesos según requisitos
criterios_base = ['distancia', 'precio_alquiler', 'precio_venta']
pesos = {k: v if (k in criterios_base or requisitos.get(k, False)) else 0 for k, v in pesos_usuario.items()}

# Funcion para calcular puntuación personalizada
def puntuacion_personalizada(row, pesos):
    score = 0
    score += pesos.get('distancia', 0) * (max(0, 100 - row['Distancia_km']) / 100)
    score += pesos.get('precio_alquiler', 0) * (max(0, 1000 - row['Precio_Alquiler']) / 1000)
    score += pesos.get('precio_venta', 0) * (max(0, 1000 - row['Precio_Venta']) / 1000)
    score += pesos.get('cap', 0) * min(1, row['Num_Centros_AP'])
    score += pesos.get('hospital', 0) * min(1, row['Num_Hospitales'])
    score += pesos.get('infantil', 0) * min(1, row['Num_Infantil'])
    score += pesos.get('primaria', 0) * min(1, row['Num_Primaria'])
    score += pesos.get('secundaria', 0) * min(1, row['Num_Secundaria'])
    score += pesos.get('entorno_rural', 0) if row['Es_rural'] else 0
    return score

# Aplicamos la función con los pesos filtrados
df_filtrado['Puntuacion'] = df_filtrado.apply(lambda row: puntuacion_personalizada(row, pesos), axis=1)

# Ordenamos por puntuación y obtener los 5 mejores municipios
df_result = df_filtrado.sort_values('Puntuacion', ascending=False).head(5)



API IDEALISTA

In [ ]:
# Función para obtener token de acceso a la API de Idealista
def get_idealista_token(client_id, client_secret):
    credentials = f"{client_id}:{client_secret}"
    b64_credentials = base64.b64encode(credentials.encode()).decode()

    auth_url = "https://api.idealista.com/oauth/token"
    auth_data = {
        "grant_type": "client_credentials",
        "scope": "read"
    }
    auth_headers = {
        "Authorization": f"Basic {b64_credentials}",
        "Content-Type": "application/x-www-form-urlencoded"
    }

    response = requests.post(auth_url, data=auth_data, headers=auth_headers)
    return response.json().get("access_token", None)  # Devolver token o None

# Función para obtener número de ofertas cerca de una ubicación (alquiler o venta)
def get_num_ofertas_idealista(lat, lon, token, tipo='rent'):
    url = "https://api.idealista.com/3.5/es/search"
    headers = {
        "Authorization": f"Bearer {token}"
    }
    params = {
        "operation": tipo,             # Tipo de operación: 'rent' o 'sale'
        "center": f"{lat},{lon}",      # Coordenadas centrales
        "distance": "2000",            # Radio en metros alrededor del punto
        "propertyType": "homes",       # Tipo de inmueble
        "maxItems": 1,
        "numPage": 1,
        "language": "es"
    }

    r = requests.post(url, headers=headers, data=params)
    if r.status_code == 200:
        return r.json().get("total", 0)  # Número total de ofertas
    else:
        return None                      # En caso de error

# Credenciales de la API
client_id = 'pd4uvk431liz60nxwryn543z6njwumvm'
client_secret = 'Zm2PkpzIIGXt'

# Obtenemos token para autenticación
token_idealista = get_idealista_token(client_id, client_secret)


Comprovamos que funciona con un ejemplo de prueba

In [ ]:
# Coordenadas de Manresa
lat_manresa = 41.7288
lon_manresa = 1.8231

# Obtenemos número de ofertas en alquiler cerca de Manresa
ofertas_alquiler = get_num_ofertas_idealista(lat_manresa, lon_manresa, token_idealista, tipo='rent')

# Obtenemos número de ofertas en venta cerca de Manresa
ofertas_venta = get_num_ofertas_idealista(lat_manresa, lon_manresa, token_idealista, tipo='sale')

# Mostramos resultados por pantalla
print(f"Ofertas alquiler en Manresa: {ofertas_alquiler}")
print(f"Ofertas venta en Manresa: {ofertas_venta}")

Ofertas alquiler en Manresa: 27
Ofertas venta en Manresa: 727


In [ ]:
# Para cada municipio recomendado, consultamos la API de Idealista
# para obtener el número de ofertas disponibles de alquiler y venta,
# almacenamos esos datos en el DataFrame y mostramos un resumen completo.

# Recorremos los municipios recomendados para obtener ofertas de Idealista
for idx, row in df_result.iterrows():
    lat = row['LATITUD_ETRS89']
    lon = row['LONGITUD_ETRS89']

    # Consultamos ofertas de alquiler y venta en Idealista
    ofertas_alquiler = get_num_ofertas_idealista(lat, lon, token_idealista, tipo='rent')
    ofertas_venta = get_num_ofertas_idealista(lat, lon, token_idealista, tipo='sale')

    # Guardamos resultados en nuevas columnas del DataFrame
    df_result.at[idx, 'Ofertas_Alquiler'] = ofertas_alquiler
    df_result.at[idx, 'Ofertas_Venta'] = ofertas_venta

    # Mostramos resumen de cada municipio
    print(f"Municipio: {row['Nombre']} ({row['Provincia']})")
    print(f" - Habitantes: {row['Habitantes']} ({'Rural' if row['Es_rural'] else 'No rural'})")
    print(f" - Distancia: {row['Distancia_km']:.2f} km, Tiempo coche: {row['Tiempo_coche_min']:.0f} min")
    print(f" - Precio Alquiler: {row['Precio_Alquiler']} € — Ofertas: {ofertas_alquiler}")
    print(f" - Precio Venta: {row['Precio_Venta']} € — Ofertas: {ofertas_venta}")
    print(f" - CAP: {row['Num_Centros_AP']}, Hospitales: {row['Num_Hospitales']}")
    print(f" - Escuelas: Infantil={row['Num_Infantil']}, Primaria={row['Num_Primaria']}, Secundaria={row['Num_Secundaria']}")
    print(f" - Puntuación: {row['Puntuacion']:.2f}")
    print('---------------------------')

Municipio: Sant Adrià de Besòs (Barcelona)
 - Habitantes: 38490.0 (No rural)
 - Distancia: 6.38 km, Tiempo coche: 4 min
 - Precio Alquiler: 14.665 € — Ofertas: 101
 - Precio Venta: 2103.261194029851 € — Ofertas: 2224
 - CAP: 2, Hospitales: 0
 - Escuelas: Infantil=14, Primaria=8, Secundaria=6
 - Puntuación: 9.74
---------------------------
Municipio: Esplugues de Llobregat (Barcelona)
 - Habitantes: 47613.0 (No rural)
 - Distancia: 7.82 km, Tiempo coche: 5 min
 - Precio Alquiler: 23.4 € — Ofertas: 165
 - Precio Venta: 3974.0 € — Ofertas: 1346
 - CAP: 2, Hospitales: 4
 - Escuelas: Infantil=23, Primaria=14, Secundaria=11
 - Puntuación: 9.69
---------------------------
Municipio: Sant Just Desvern (Barcelona)
 - Habitantes: 20881.0 (No rural)
 - Distancia: 8.37 km, Tiempo coche: 5 min
 - Precio Alquiler: 17.4 € — Ofertas: 139
 - Precio Venta: 4129.0 € — Ofertas: 403
 - CAP: 1, Hospitales: 0
 - Escuelas: Infantil=10, Primaria=6, Secundaria=4
 - Puntuación: 9.67
---------------------------
M

In [ ]:
# Creamos el mapa centrado en el municipio origen
mapa = folium.Map(location=[lat_origen, lon_origen], zoom_start=9)

# Añadimos marcador para el municipio origen en rojo
folium.Marker(
    location=[lat_origen, lon_origen],
    popup=f"Origen: {localidad_match}",
    icon=folium.Icon(color='red', icon='home')
).add_to(mapa)

# Creamos grupo para agrupar marcadores cercanos
marker_cluster = MarkerCluster().add_to(mapa)

# Añadimos marcadores para municipios recomendados con info detallada
for idx, row in df_result.iterrows():
    popup_text = f"""
    <b>{row['Nombre']} ({row['Provincia']})</b><br>
    Habitantes: {row['Habitantes']}<br>
    Distancia: {row['Distancia_km']:.2f} km<br>
    Alquiler: {row['Precio_Alquiler']} € — Ofertas: {row['Ofertas_Alquiler']}<br>
    Venta: {row['Precio_Venta']} € — Ofertas: {row['Ofertas_Venta']}<br>
    CAP: {row['Num_Centros_AP']} | Hospitales: {row['Num_Hospitales']}<br>
    Escuelas: I={row['Num_Infantil']} P={row['Num_Primaria']} S={row['Num_Secundaria']}<br>
    Puntuación: {row['Puntuacion']:.2f}
    """
    folium.Marker(
        location=[row['LATITUD_ETRS89'], row['LONGITUD_ETRS89']],
        popup=popup_text,
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(marker_cluster)

# Mostramos el mapa generado
mapa


Finalmente lo viusalizamos con una demo interactiva

In [ ]:
# Reescribimos todo el código para crear una demo con Gradio
# Carga de datos
df = pd.read_excel("general_7.xlsx")
df['Habitantes'] = pd.to_numeric(df['Habitantes'], errors='coerce')

# API Idealista
def get_idealista_token(client_id, client_secret):
    credentials = f"{client_id}:{client_secret}"
    b64_credentials = base64.b64encode(credentials.encode()).decode()
    auth_url = "https://api.idealista.com/oauth/token"
    headers = {
        "Authorization": f"Basic {b64_credentials}",
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {
        "grant_type": "client_credentials",
        "scope": "read"
    }
    r = requests.post(auth_url, headers=headers, data=data)
    return r.json().get("access_token")

def get_num_ofertas_idealista(lat, lon, token, tipo='rent'):
    url = "https://api.idealista.com/3.5/es/search"
    headers = {"Authorization": f"Bearer {token}"}
    params = {
        "operation": tipo,
        "center": f"{lat},{lon}",
        "distance": "2000",
        "propertyType": "homes",
        "maxItems": 1,
        "numPage": 1,
        "language": "es"
    }
    r = requests.post(url, headers=headers, data=params)
    if r.status_code == 200:
        return r.json().get("total", 0)
    return None

# Credenciales
client_id = 'pd4uvk431liz60nxwryn543z6njwumvm'
client_secret = 'Zm2PkpzIIGXt'

# Función principal
def demo_modelo(localidad_input, provincia_input, modalidad_trabajo, distancia_max_km,
                alquiler_max, venta_max, cap, hospital, infantil, primaria, secundaria, rural,
                peso_dist, peso_alq, peso_venta, peso_cap, peso_hosp, peso_inf, peso_prim, peso_sec, peso_rural):

    token_idealista = get_idealista_token(client_id, client_secret)

    requisitos = {
        'cap': cap,
        'hospital': hospital,
        'infantil': infantil,
        'primaria': primaria,
        'secundaria': secundaria,
        'entorno_rural': rural
    }

    pesos_usuario = {
        'distancia': peso_dist,
        'precio_alquiler': peso_alq,
        'precio_venta': peso_venta,
        'cap': peso_cap,
        'hospital': peso_hosp,
        'infantil': peso_inf,
        'primaria': peso_prim,
        'secundaria': peso_sec,
        'entorno_rural': peso_rural
    }

    def haversine(lon1, lat1, lon2, lat2):
        lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
        c = 2 * asin(sqrt(a))
        return 6371 * c

    df_prov = df[df['Provincia'].str.lower() == provincia_input.lower()]
    if df_prov.empty:
        return f"<p style='color:red;'>Provincia '{provincia_input}' no encontrada.</p>", pd.DataFrame()

    municipios = df_prov['Nombre'].tolist()
    localidad_match, _ = process.extractOne(localidad_input, municipios)
    origen = df_prov[df_prov['Nombre'] == localidad_match].iloc[0]
    lon_origen = origen['LONGITUD_ETRS89']
    lat_origen = origen['LATITUD_ETRS89']

    df['Distancia_km'] = df.apply(lambda row: haversine(lon_origen, lat_origen, row['LONGITUD_ETRS89'], row['LATITUD_ETRS89']), axis=1)
    df['Es_rural'] = df['Habitantes'] < 10000

    df_filtrado = df[
        (df['Distancia_km'] <= distancia_max_km) &
        (df['Precio_Alquiler'] <= alquiler_max) &
        (df['Precio_Venta'] <= venta_max) &
        (df['Nombre'].str.lower() != localidad_match.lower())
    ]

    if requisitos['cap']: df_filtrado = df_filtrado[df_filtrado['Num_Centros_AP'] > 0]
    if requisitos['hospital']: df_filtrado = df_filtrado[df_filtrado['Num_Hospitales'] > 0]
    if requisitos['infantil']: df_filtrado = df_filtrado[df_filtrado['Num_Infantil'] > 0]
    if requisitos['primaria']: df_filtrado = df_filtrado[df_filtrado['Num_Primaria'] > 0]
    if requisitos['secundaria']: df_filtrado = df_filtrado[df_filtrado['Num_Secundaria'] > 0]
    if requisitos['entorno_rural']: df_filtrado = df_filtrado[df_filtrado['Es_rural'] == True]

    def puntuacion_personalizada(row):
        score = 0
        score += pesos_usuario.get('distancia', 0) * (max(0, 100 - row['Distancia_km']) / 100)
        score += pesos_usuario.get('precio_alquiler', 0) * (max(0, 1000 - row['Precio_Alquiler']) / 1000)
        score += pesos_usuario.get('precio_venta', 0) * (max(0, 1000 - row['Precio_Venta']) / 1000)
        score += pesos_usuario.get('cap', 0) * min(1, row['Num_Centros_AP'])
        score += pesos_usuario.get('hospital', 0) * min(1, row['Num_Hospitales'])
        score += pesos_usuario.get('infantil', 0) * min(1, row['Num_Infantil'])
        score += pesos_usuario.get('primaria', 0) * min(1, row['Num_Primaria'])
        score += pesos_usuario.get('secundaria', 0) * min(1, row['Num_Secundaria'])
        score += pesos_usuario.get('entorno_rural', 0) if row['Es_rural'] else 0
        return score

    df_filtrado['Puntuacion'] = df_filtrado.apply(puntuacion_personalizada, axis=1)
    df_top = df_filtrado.sort_values('Puntuacion', ascending=False).head(5)

    df_top['Ofertas_Alquiler'] = df_top.apply(lambda row: get_num_ofertas_idealista(row['LATITUD_ETRS89'], row['LONGITUD_ETRS89'], token_idealista, 'rent'), axis=1)
    df_top['Ofertas_Venta'] = df_top.apply(lambda row: get_num_ofertas_idealista(row['LATITUD_ETRS89'], row['LONGITUD_ETRS89'], token_idealista, 'sale'), axis=1)

    mapa = folium.Map(location=[lat_origen, lon_origen], zoom_start=9)
    folium.Marker([lat_origen, lon_origen], popup=f"Origen: {localidad_match}", icon=folium.Icon(color='red')).add_to(mapa)
    cluster = MarkerCluster().add_to(mapa)

    for _, row in df_top.iterrows():
        popup = f"<b>{row['Nombre']} ({row['Provincia']})</b><br>"
        popup += f"Habitantes: {row['Habitantes']}<br>Distancia: {row['Distancia_km']:.1f} km<br>"
        popup += f"Alquiler: {row['Precio_Alquiler']} € (Ofertas: {row['Ofertas_Alquiler']})<br>"
        popup += f"Venta: {row['Precio_Venta']} € (Ofertas: {row['Ofertas_Venta']})<br>"
        popup += f"CAP: {row['Num_Centros_AP']} | Hosp: {row['Num_Hospitales']}<br>"
        popup += f"Escuelas: I={row['Num_Infantil']} P={row['Num_Primaria']} S={row['Num_Secundaria']}<br>"
        popup += f"Puntuación: {row['Puntuacion']:.2f}"
        folium.Marker([row['LATITUD_ETRS89'], row['LONGITUD_ETRS89']], popup=popup).add_to(cluster)

    return mapa._repr_html_(), df_top[['Nombre', 'Provincia', 'Precio_Alquiler', 'Precio_Venta', 'Ofertas_Alquiler', 'Ofertas_Venta', 'Puntuacion']]

# Gradio Interface
iface = gr.Interface(
    fn=demo_modelo,
    inputs=[
        gr.Textbox(label="Localidad origen", value="Barcelona"),
        gr.Textbox(label="Provincia", value="Barcelona"),
        gr.Radio(["remoto", "híbrido"], label="Modalidad laboral"),
        gr.Slider(10, 300, step=10, label="Distancia máxima (km)", value=200),
        gr.Slider(0, 30, step=1, label="Alquiler máx. €/m²", value=15),
        gr.Slider(0, 5000, step=100, label="Venta máx. €/m²", value=1500),
        gr.Checkbox(label="Centro de atención primaria"),
        gr.Checkbox(label="Hospital"),
        gr.Checkbox(label="Escuela infantil"),
        gr.Checkbox(label="Primaria"),
        gr.Checkbox(label="Secundaria"),
        gr.Checkbox(label="¿Prefieres entorno rural?"),
        gr.Slider(0, 5, step=1, label="Peso distancia"),
        gr.Slider(0, 5, step=1, label="Peso alquiler"),
        gr.Slider(0, 5, step=1, label="Peso venta"),
        gr.Slider(0, 5, step=1, label="Peso CAP"),
        gr.Slider(0, 5, step=1, label="Peso hospital"),
        gr.Slider(0, 5, step=1, label="Peso infantil"),
        gr.Slider(0, 5, step=1, label="Peso primaria"),
        gr.Slider(0, 5, step=1, label="Peso secundaria"),
        gr.Slider(0, 5, step=1, label="Peso ruralidad"),
    ],
    outputs=[
        gr.HTML(label="Mapa interactivo"),
        gr.Dataframe(label="Top 5 municipios recomendados")
    ],
    title="Recomendador de Municipios para Teletrabajo",
    description="Encuentra el mejor municipio para mudarte según tus necesidades y prioridades."
)

iface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://410387aebb31c8a263.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
